# 基本的图算法

In [2]:
import Queue

class Node:
    def __init__(self, value=None):
        self.value = value
        # color is "W" for white, "G" for gray, "B" for black
        self.color = "W"
        #depth means the length to root in breadth_first_search
        #and find time in depth_first_search
        self.depth = None
        self.final = None #final time used in depth_first_search
        self.pre = None
    def reset(self):
        self.color = "W"
        self.depth = None
        self.final = None #final time used in depth_first_search
        self.pre = None
        
#通过领接链表保存图
class Graphy:
    def __init__(self):
        self.node_neighbors = {} #map list
        self.visited = {}
        self.time = 0
    
    def add_nodes(self, nodelist):
        for node in nodelist:
            self.add_node(node)
    
    def add_node(self, node):
        if not node in self.nodes():
            self.node_neighbors[node] = []
    
    def add_edge(self, edge, bDi=True):
        u,v = edge
        if(v not in self.node_neighbors[u]):
            self.node_neighbors[u].append(v)
        if(False == bDi):
            if(u not in self.node_neighbors[v]):
                self.node_neighbors[v].append(u)

    def nodes(self):
        return self.node_neighbors.keys()
    
    #有向图的转置
    def transposition_graphy(self):
        g = Graphy()
        g.add_nodes(self.node_neighbors.keys())
        for key in self.node_neighbors.keys():
            for node in self.node_neighbors[key]:
                g.add_edge((node,key))
        return g
    
    #广度优先遍历
    def breadth_first_search(self, s):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.pre = None
        s.color = "G"
        s.depth = 0
        s.pre = None
        Q = Queue.Queue()
        Q.put(s)
        while(not Q.empty()):
            u = Q.get()
            for v in self.node_neighbors[u]:
                if("W" == v.color):
                    v.color = "G"
                    v.depth = u.depth+1
                    v.pre = u
                    Q.put(v)
            u.color = "B"

    #深度优先遍历
    def depth_first_search(self):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.final = None
            node.pre = None
        self.time = 0
        for node in self.node_neighbors.keys():
            if("W" == node.color):
                self.dfs_visit(node)
    def dfs_visit(self, u):
        self.time += 1
        u.depth = self.time
        u.color = "G"
        for node in self.node_neighbors[u]:
            if("W" == node.color):
                node.pre = u
                self.dfs_visit(node)
        u.color = "B"
        self.time += 1
        u.final = self.time
        
    #拓扑排序    
    def topological_sort(self, isSort=True):
        if(True == isSort):
            self.depth_first_search()
        nodelist = []
        insert_pos = 0
        for node in self.node_neighbors.keys():
            for insert_pos in range(0,len(nodelist)+1):
                if(insert_pos >= len(nodelist)):
                    break;
                if(node.final > nodelist[insert_pos].final):
                    break
            nodelist.insert(insert_pos, node)
        return nodelist
    
    def strongly_connected_components(self):
        self.depth_first_search()
        node_list = self.nodes()
        g = self.transposition_graphy()
        nodelist = g.topological_sort(False)
        tree = []
        g.time = 0
        for node in nodelist:
            node.reset()
        for node in nodelist:
            if("W" == node.color):
                g.dfs_visit(node)
                tree.append(node)
        return tree
        
    
g = Graphy()
nodes = [Node(i) for i in range(8)]
g.add_nodes(nodes)
g.add_edge((nodes[0], nodes[1]))
g.add_edge((nodes[1], nodes[2]))
g.add_edge((nodes[1], nodes[5]))
g.add_edge((nodes[1], nodes[4]))
g.add_edge((nodes[2], nodes[3]))
g.add_edge((nodes[2], nodes[6]))
g.add_edge((nodes[3], nodes[7]))
g.add_edge((nodes[3], nodes[2]))
g.add_edge((nodes[4], nodes[0]))
g.add_edge((nodes[4], nodes[5]))
g.add_edge((nodes[5], nodes[6]))
g.add_edge((nodes[6], nodes[5]))
g.add_edge((nodes[6], nodes[7]))

print("breadth first search:")
g.breadth_first_search(nodes[0])
for node in g.nodes():
    print (node.value, node.color, node.depth)

print("depth first search:")
g.depth_first_search()
node_list = g.topological_sort()
for node in node_list:
    print (node.value, node.color, node.depth, node.final)

print("nums of strongly connected components:")
treelist = g.strongly_connected_components()
print(len(treelist))


breadth first search:
(2, 'B', 2)
(4, 'B', 2)
(5, 'B', 2)
(6, 'B', 3)
(7, 'B', 4)
(0, 'B', 0)
(3, 'B', 3)
(1, 'B', 1)
depth first search:
(4, 'B', 11, 16)
(0, 'B', 12, 15)
(1, 'B', 13, 14)
(2, 'B', 1, 10)
(6, 'B', 6, 9)
(5, 'B', 7, 8)
(3, 'B', 2, 5)
(7, 'B', 3, 4)
nums of strongly connected components:
4


# 最小生成树(Kruskal算法)

In [2]:
import numpy as np
import Queue

# 互斥集，用于实现节点的union和find
# http://blog.csdn.net/polly_yang/article/details/8198604
class DisjointSets:
    def __init__(self):
        self.parent = {}
    def add(self, nodes):
        for node in nodes:
            self.parent[node] = node
    def find(self, item):
        parent = self.parent[item]
        while(self.parent[parent] != parent):
            parent = self.parent[parent]
        self.parent[item] = parent
        return parent
    def union(self,item1, item2):
        self.parent[item1] = self.parent[item2]
        
class PrimMST:
    def __init__(self, G):
        self.G = G
        self.edgeTo = []*G[0].size   #距离树最近的边
        self.distTo = []*G[0].size   #distTo[w] 对应于edgeTo[w]的边
        self.marked = []*G[0].size   #v在树中则为True
        self.minWeight = (0, 0)    #第一个是点，第二个是权值
    
    def prim(self):
        #最终得到的结果是self.edgeTo中的边和self.distTo中的权值
        for i in range(self.G[0].size):
            self.edgeTo[i] = None
            self.distTo[i] = sys.maxint
            self.marked[i] = False
        distTo[0] = 0
        for i in range(self.G[0].size):
            visit(self.minWeight[0])
        return (self.edgeTo, self.distTo)
        
    def visit(self, v):
        # 将顶点v添加到树中
        minW = sys.maxint
        self.marked[v] = True
        for w in G[v].size:
            weight = G[v][w]
            if(0 == weight or True == self.marked[w]):
                continue
            if(weight < distTo[w]):
                edgeTo[w] = (v,w)
                distTo[w] = weight
                # 将w和weight插入一个堆中，关键是这个堆如何实现?
                # 需要按weight排序，又要按
                if(weight < minW):
                    self.minWeight = (w, weight)
        
# 通过领接矩阵保存图
class UndiGraph:
    def __init__(self):
        self.nodes = []
        self.matrix = None
        self.edge_list = []
        self.forests = DisjointSets()
    
    def create_matrix(self, nodes):
        self.nodes = nodes
        self.forests.add(nodes)
        len(nodes)
        self.matrix = np.zeros([len(nodes), len(nodes)], dtype = np.int16)
    
    # 无向图
    def add_edge(self, edge):
        u,v,weight = edge
        if(u == v):
            return
        if((u in self.nodes) and (v in self.nodes)):
            self.edge_list.append((u, v,weight))
            row = self.nodes.index(u)
            col = self.nodes.index(v)
            self.matrix[row, col] = weight
            self.matrix[col, row] = weight
    
    def sort_edge_list(self):
        def cmp(e1,e2):
            if(e1[2] < e2[2]):
                return -1
            elif(e1[2] > e2[2]):
                return 1
            else:
                return 0
        self.edge_list.sort(cmp)
    
    def mst_kruskal(self):
        self.sort_edge_list()
        accpeted_e_list=[]
        for edge in self.edge_list:
            u = edge[0]
            v = edge[1]
            uset=self.forests.find(u)
            vset=self.forests.find(v)
            if(uset!=vset):
                accpeted_e_list.append(edge)
                self.forests.union(uset,vset)
        return accpeted_e_list
    
    def mst_prim(self):
        prim = PrimMST(self.matrix)
        return prim.prim()
        
if __name__=='__main__':
    nodes = ['a','b','c','d','e','f','g','h','i']
    ug = UndiGraph()
    ug.create_matrix(nodes)
    ug.add_edge(('a','b',4))
    ug.add_edge(('a','h',8))
    ug.add_edge(('b','c',8))
    ug.add_edge(('b','h',11))
    ug.add_edge(('c','d',7))
    ug.add_edge(('c','f',4))
    ug.add_edge(('c','i',2))
    ug.add_edge(('d','e',9))
    ug.add_edge(('d','f',14))
    ug.add_edge(('e','f',10))
    ug.add_edge(('f','g',2))
    ug.add_edge(('g','h',1))
    ug.add_edge(('g','i',6))
    ug.add_edge(('h','i',7))
    
    arr = ug.mst_kruskal()
    #print(arr)
    
    (edgeArr, distArr) = ug.mst_prim()
    print(edgeArr)
    print(distArr)


IndexError: list assignment index out of range